In [ ]:
# import functools
#
# def config_instr(exec_after: int):
#     # Python's lazy interpreter makes a 'for' loop fail to update its index inside a closure. This 'nonlocal i' workaround is sufficient because the decorated functions are called in strict sequence. If the CPU had to simulate multithreading, this would probably break, too.
#     i = 0
#
#     def instr_decorator(instr_f):
#
#         @functools.wraps(instr_f)
#         def instr_wrapper(*args, **kwargs):
#             nonlocal i
#             i += 1
#             if i == exec_after:
#                 i = 0  # reset
#                 return instr_f(*args, **kwargs)
#             else:
#                 return False
#
#         return instr_wrapper
#
#     return instr_decorator
#
#
# @config_instr(exec_after=1)
# def noop():
#     return True
#
#
# @config_instr(exec_after=3)
# def add(reg: int, val: int):
#     # TODO OMG when starting this problem I thought 'reg' would be updated in the scope where it was declared! After hours of debugging and Googling I've learned that no, that's not how it works. Duh! I probably didn't need any of this complicated design! But I did learn how closures and decorators work in Python, and some new things about lazy object assignment.
#     # reg += val
#     return reg + val


def delay(instruction: str):
    match instruction:
        case ['noop']:
            delayed = (_ for _ in [True])
        case ['addx', *others]:
            delayed = (_ for _ in (False, False, True))  # Refresh the delay generator
        case [cmd, *others]:
                raise ValueError(f'{cmd} with args {others} not supported.')
    return delayed


def read_instr(file) -> list:
    return f.readline().split()


with open('../inputs/day10-input') as f:

    """ This loop is the CPU """
    # Init state
    cycle = 0  #  Required for 'signal_strength' math
    reg_x = 1
    signal_strengths = []
    all_done = False

    # Load first instruction
    instr = read_instr(f)
    delayed_exec = delay(instr)

    while True:
        """
        CPU operation order:
        1. Update clock cycle
        2. Measure signal strength
        3. Execute current instruction (might be delayed/continued from last cycle)
        4. If instruction complete, read another
        5. If objectives are met, quit
        """
        cycle += 1

        if (cycle - 20) % 40 == 0:
            signal_strengths.append(cycle * reg_x)
            print(f'{cycle}: Signal strength = {signal_strengths[-1]}')  # debug

            if len(signal_strengths) == 6:
                all_done = True  # Part 1 puzzle complete

        print(f'{cycle}: {" ".join(instr)}', end='')  # debug
        if next(delayed_exec):
            match instr:
                case ['noop']:
                    done = True
                case ['addx', value]:
                    reg_x += int(value)
                    done = True
                case [cmd, *others]:
                    raise ValueError(f'{cmd} with args {others} not supported.')

        print(f' | X = {reg_x}')  # debug

        if done:
            instr = read_instr(f)
            delayed_exec = delay(instr)
            if not instr:
                all_done = True  # Instruction list complete
            done = False

        if all_done:
            break


print(f'Signal strengths: {signal_strengths} // Total: {sum(signal_strengths)}')
# TODO 15660 is too high